In [13]:
import time
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import train_test_split

In [14]:
more_features = False

train_df = pd.read_csv('input/train' + ('_more_features' if more_features else '') + '.csv')
test_df = pd.read_csv('input/test'  + ('_more_features' if more_features else '') + '.csv')
    
X_test = test_df.drop('ID_code',axis=1)
X = train_df.drop(['ID_code','target'],axis=1)
y = train_df['target']

In [15]:
params = {
    'metric': 'auc',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'min_data_in_leaf': 2881,
    'max_depth': 0,
    'num_leaves': 3,
    'learning_rate': 0.01,
    'bagging_freq': 3,
    #'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.3, 0.9),
    'feature_fraction': 0.8453828656355421,
    'bagging_seed': 11,
    'reg_alpha':  1.1173044727720816,
    'reg_lambda': 6.9285776442737514,
    'random_state': 42,
    'verbosity': -1,
    'subsample':0.8421287738494433,
    'min_child_weight': 36.93038816860224,
    'num_threads': 4,
    'max_bin': 483
}

In [16]:
score = 0.0
prediction = np.zeros(len(X_test))

In [17]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=10)

In [18]:
train_data = lgb.Dataset(X_train, label=y_train)
valid_data = lgb.Dataset(X_valid, label=y_valid)

model = lgb.train(params,train_data,num_boost_round=2000000,
                  valid_sets = [train_data, valid_data],verbose_eval=300,early_stopping_rounds = 800)

prediction += model.predict(X_test, num_iteration=model.best_iteration)
score += model.best_score['valid_1']['auc']

print(score)

Training until validation scores don't improve for 800 rounds.
[300]	training's auc: 0.767414	valid_1's auc: 0.76129
[600]	training's auc: 0.803213	valid_1's auc: 0.796306
[900]	training's auc: 0.823591	valid_1's auc: 0.816502
[1200]	training's auc: 0.837672	valid_1's auc: 0.830386
[1500]	training's auc: 0.84824	valid_1's auc: 0.840975
[1800]	training's auc: 0.856153	valid_1's auc: 0.848732
[2100]	training's auc: 0.862805	valid_1's auc: 0.855181
[2400]	training's auc: 0.86778	valid_1's auc: 0.86005
[2700]	training's auc: 0.872276	valid_1's auc: 0.864359
[3000]	training's auc: 0.876125	valid_1's auc: 0.868141
[3300]	training's auc: 0.879587	valid_1's auc: 0.871474
[3600]	training's auc: 0.882359	valid_1's auc: 0.874098
[3900]	training's auc: 0.885032	valid_1's auc: 0.876536
[4200]	training's auc: 0.887409	valid_1's auc: 0.878779
[4500]	training's auc: 0.889565	valid_1's auc: 0.880814
[4800]	training's auc: 0.891444	valid_1's auc: 0.882562
[5100]	training's auc: 0.893231	valid_1's auc: 0

In [19]:
sub = pd.DataFrame({"ID_code": test_df.ID_code.values})
sub["target"] = prediction
sub.to_csv("submission.csv", index=False)